In [1]:
workspace_id = "4bc212c2-fb47-41dd-bb4c-95c4927ca2e8"
a_report_id = "678f6aff-94f7-474d-815e-6e86f4560927"
a = 'Industry Margin Analysis'

import requests
from pandas.api.types import is_numeric_dtype
import numpy as np
import sys
from powerbiclient import Report, models
from io import StringIO
from ipywidgets import interact
import requests
import pandas as pd
from tabulate import tabulate

from powerbiclient.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication()
token_with_redirect = interactive_auth.get_access_token()

a_report = Report(group_id=workspace_id, report_id=a_report_id, auth=interactive_auth)
a_report


def get_page_id(page_name, report_obj):
    for page in report_obj.get_pages():
        if page['displayName'] == str(page_name):
            return(page['name'])
        
def get_visual_id(page_id, visual_name, report_obj):
    for vis in report_obj.visuals_on_page(str(page_id)):
        try:
            if vis['title'] == str(visual_name):
                return(vis['name'])
        except:
            pass
        
def get_visual_info(page_name, visual_name, report_obj):
    page_id = get_page_id(str(page_name),report_obj)
    visual_id = get_visual_id(page_id,str(visual_name),report_obj)
    df = pd.read_csv(StringIO(report_obj.export_visual_data(str(page_id), str(visual_id))), sep=",")
    return(df)

a_page_name_1 = 'Team Scorecard'
a_visual_name_1 = 'Scorecard by Executive'
a_cols_1 = ["Executive Name", "# of Customers", "# of Products", "YTD Revenue", "YTD GM%", "YTD COGS", "Sum of Taxes"]

a_df = get_visual_info(a_page_name_1, a_visual_name_1, a_report)
a_df

In [2]:
from powerbiclient.authentication import DeviceCodeLoginAuthentication
device_auth = DeviceCodeLoginAuthentication()
token_with_device_code = device_auth.get_access_token()

Performing interactive authentication. Please follow the instructions on the terminal.
 To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code HF3KA8BRM to authenticate.
You have logged in.
Interactive authentication successfully completed.


In [16]:
workspace_id = "4bc212c2-fb47-41dd-bb4c-95c4927ca2e8"
a_report_id = "678f6aff-94f7-474d-815e-6e86f4560927"
b_report_id = "d05ecb47-d423-487d-887d-0e4d000e5e27"
a = 'Industry Margin Analysis'
b = 'Customer Profitability'

a_page_name_1 = 'Team Scorecard'
a_visual_name_1 = 'Scorecard by Executive'
a_cols_1 = ["Executive Name", "# of Customers", "# of Products", "YTD Revenue", "YTD GM%", "YTD COGS", "Sum of Taxes"]

b_page_name_1 = 'Industry Margin Analysis'
b_visual_name_1 = 'Executive Table'
b_cols_1 = ["Name", "# of Customers", "# of Products", "YTD Revenue", "YTD GM%", "YTD COGS", "Sum of Taxes"]

import requests
from pandas.api.types import is_numeric_dtype
import numpy as np
import sys
from powerbiclient import Report, models
from io import StringIO
from ipywidgets import interact
import requests
import pandas as pd
from tabulate import tabulate

from powerbiclient.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication()
token_with_redirect = interactive_auth.get_access_token()

a_report = Report(group_id=workspace_id, report_id=a_report_id, auth=interactive_auth)
a_report

b_report = Report(group_id=workspace_id, report_id=b_report_id, auth=interactive_auth)
b_report

def get_page_id(page_name, report_obj):
    for page in report_obj.get_pages():
        if page['displayName'] == str(page_name):
            return(page['name'])
        
def get_visual_id(page_id, visual_name, report_obj):
    for vis in report_obj.visuals_on_page(str(page_id)):
        try:
            if vis['title'] == str(visual_name):
                return(vis['name'])
        except:
            pass
        
def get_visual_info(page_name, visual_name, report_obj):
    page_id = get_page_id(str(page_name),report_obj)
    visual_id = get_visual_id(page_id,str(visual_name),report_obj)
    df = pd.read_csv(StringIO(report_obj.export_visual_data(str(page_id), str(visual_id))), sep=",")
    return(df)

def format_numeric(df):
    objectCols = [col for col, col_type in df.dtypes.iteritems() if col_type == 'object']
    if len(objectCols) > 0:
        df[objectCols] = df[objectCols].apply(pd.to_numeric, errors='ignore')
        numericCols = list(df.select_dtypes(include=[float, np.float64, np.float16, np.float32, int, np.int8, np.int16, np.int32, np.int64]).columns)
        df[numericCols] = df[numericCols].astype(np.float64)
    return df 

def clean_data(df):
    for col in list(set(df.columns)-set(df._get_numeric_data().columns)):
        exec("""df['{}'] = df['{}'].str.rstrip('%')""".format(col,col))
        exec("""df['{}'] = df['{}'].str.lstrip('$')""".format(col,col))
    return(format_numeric(df).round())

def compare_metrics(df1, df2, ref_col):
    df2.columns = df1.columns
    df1 = df1.sort_values(str(ref_col))
    df1 = df1.reset_index(drop = True)
    df1.dropna(inplace = True, how = 'all')
    df1.fillna(0, inplace = True)
    df2 = df2.sort_values(str(ref_col))
    df2 = df2.reset_index(drop = True)
    df2.dropna(inplace = True, how = 'all')
    df2.fillna(0, inplace = True)
    for col in df1.columns:
        if sum(np.where(df1[str(col)] == df2[str(col)], 0, 1)) > 0:
            print(f"\nThe {a}'s {col} values of {df1[str(ref_col)].iloc[np.where(df1[str(col)] != df2[str(col)])].values} are not tying out with {b}'s")
            
a_df = get_visual_info(a_page_name_1, a_visual_name_1, a_report)
a_df = a_df[a_cols_1]
a_df = clean_data(a_df)
b_df = get_visual_info(b_page_name_1, b_visual_name_1, b_report)
b_df = b_df[b_cols_1]
b_df = clean_data(b_df)
b_df.columns = a_cols_1
compare_metrics(a_df, b_df, 'Executive Name')

A local browser window will be open for interactive sign in.
You have logged in.
Interactive authentication successfully completed.


In [17]:
a_report = Report(group_id=workspace_id, report_id=a_report_id, auth=interactive_auth)
a_report

Report()

ConnectionError: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /organizations/v2.0/.well-known/openid-configuration (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000019C048ADE20>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [23]:
b_report = Report(group_id=workspace_id, report_id=b_report_id, auth=interactive_auth)
b_report

Report()

ConnectionError: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /organizations/v2.0/.well-known/openid-configuration (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000019C042569A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [19]:
def get_page_id(page_name, report_obj):
    for page in report_obj.get_pages():
        if page['displayName'] == str(page_name):
            return(page['name'])
        
def get_visual_id(page_id, visual_name, report_obj):
    for vis in report_obj.visuals_on_page(str(page_id)):
        try:
            if vis['title'] == str(visual_name):
                return(vis['name'])
        except:
            pass
        
def get_visual_info(page_name, visual_name, report_obj):
    page_id = get_page_id(str(page_name),report_obj)
    visual_id = get_visual_id(page_id,str(visual_name),report_obj)
    df = pd.read_csv(StringIO(report_obj.export_visual_data(str(page_id), str(visual_id))), sep=",")
    return(df)

def format_numeric(df):
    objectCols = [col for col, col_type in df.dtypes.iteritems() if col_type == 'object']
    if len(objectCols) > 0:
        df[objectCols] = df[objectCols].apply(pd.to_numeric, errors='ignore')
        numericCols = list(df.select_dtypes(include=[float, np.float64, np.float16, np.float32, int, np.int8, np.int16, np.int32, np.int64]).columns)
        df[numericCols] = df[numericCols].astype(np.float64)
    return df 

def clean_data(df):
    for col in list(set(df.columns)-set(df._get_numeric_data().columns)):
        exec("""df['{}'] = df['{}'].str.rstrip('%')""".format(col,col))
        exec("""df['{}'] = df['{}'].str.lstrip('$')""".format(col,col))
    return(format_numeric(df).round())

def compare_metrics(df1, df2, ref_col):
    df2.columns = df1.columns
    df1 = df1.sort_values(str(ref_col))
    df1 = df1.reset_index(drop = True)
    df1.dropna(inplace = True, how = 'all')
    df1.fillna(0, inplace = True)
    df2 = df2.sort_values(str(ref_col))
    df2 = df2.reset_index(drop = True)
    df2.dropna(inplace = True, how = 'all')
    df2.fillna(0, inplace = True)
    for col in df1.columns:
        if sum(np.where(df1[str(col)] == df2[str(col)], 0, 1)) > 0:
            print(f"\nThe {a}'s {col} values of {df1[str(ref_col)].iloc[np.where(df1[str(col)] != df2[str(col)])].values} are not tying out with {b}'s")

In [28]:
a_df = get_visual_info(a_page_name_1, a_visual_name_1, a_report)
a_df = a_df[a_cols_1]
a_df = clean_data(a_df)
b_df = get_visual_info(b_page_name_1, b_visual_name_1, b_report)
b_df = b_df[b_cols_1]
b_df = clean_data(b_df)
b_df.columns = a_cols_1
compare_metrics(a_df, b_df, 'Executive Name')


The Industry Margin Analysis's # of Customers values of ['Andrew Ma' 'Valery Ushakov'] are not tying out with Customer Profitability's

The Industry Margin Analysis's # of Products values of ['Andrew Ma' 'Valery Ushakov'] are not tying out with Customer Profitability's

The Industry Margin Analysis's YTD Revenue values of ['Andrew Ma' 'Valery Ushakov'] are not tying out with Customer Profitability's

The Industry Margin Analysis's YTD COGS values of ['Andrew Ma'] are not tying out with Customer Profitability's


In [9]:
a_df

,Sales_Area,CCAs QTD,CCA Attn,CCA QoQ %,CCA YoY %,NR Attn,iTero Quota,iTero Actuals,iTero % Completion
0,"Lindberg, Joanne (AM-US-WE)",33319.0,88.0,-3.0,-5.0,88.0,7983368.0,3226065.0,40.0
1,"Stemerick, Tony (AM-US-MA)",33561.0,93.0,2.0,-4.0,91.0,7221703.0,3033539.0,42.0
2,"Howard, Crystal (AM-US-NW)",27807.0,92.0,0.0,-8.0,92.0,6109568.0,3328206.0,54.0
3,"Smith, Chuck (AM-US-MW)",31837.0,93.0,5.0,-12.0,91.0,7789751.0,3076628.0,40.0
4,"Ghiz, Joe (AM-US-SW)",30167.0,90.0,1.0,3.0,91.0,7428429.0,3371450.0,45.0
5,"McNulty, Jessica (AM-US-NE)",35917.0,89.0,-2.0,-10.0,90.0,9103511.0,3459103.0,38.0
6,"Braud, Kent Park (AM-US-SE)",41279.0,95.0,3.0,0.0,94.0,9267484.0,3705956.0,40.0
7,"Accounts, House ASD (AM-US-HA-ASD)",10.0,58.0,NaN,NaN,11.0,0.0,0.0,NaN
8,"Accounts, University ASD (AM-US-UNI-ASD)",2088.0,165.0,-5.0,-0.0,87.0,0.0,0.0,NaN


In [10]:
b_df

,Sales_Area,CCAs QTD,CCA Attn,CCA QoQ %,CCA YoY %,NR Attn,iTero Quota,iTero Actuals,iTero % Completion
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Accounts, House ASD (AM-US-HA-ASD)",10.0,59.0,0.0,0.0,11.0,0.0,0.0,0.0
2,"Accounts, University ASD (AM-US-UNI-ASD)",2088.0,167.0,89.0,99.0,87.0,0.0,0.0,0.0
3,"McNulty, Jessica (AM-US-NE)",35917.0,89.0,-2.0,-10.0,90.0,9103511.0,3459103.0,38.0
4,"Braud, Kent Park (AM-US-SE)",41279.0,95.0,3.0,0.0,94.0,9267484.0,3705956.0,40.0
5,"Lindberg, Joanne (AM-US-WE)",33319.0,88.0,-3.0,-5.0,88.0,7983368.0,3226065.0,40.0
6,"Stemerick, Tony (AM-US-MA)",33561.0,93.0,2.0,-4.0,91.0,7221703.0,3033539.0,42.0
7,"Smith, Chuck (AM-US-MW)",31837.0,93.0,5.0,-12.0,91.0,7789751.0,3076628.0,40.0
8,"Howard, Crystal (AM-US-NW)",27807.0,92.0,0.0,-8.0,91.0,6109568.0,3328206.0,54.0
9,"Ghiz, Joe (AM-US-SW)",30167.0,90.0,1.0,2.0,91.0,7428429.0,3371450.0,45.0
